In [1]:
# import logging
import sys

# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd

# from JSONData import tdx_hdf5_api as h5a

# sys.stdout=stdout

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio

nest_asyncio.apply()

In [2]:
block_path = tdd.get_tdx_dir_blocknew() + '060.blk'

In [3]:
def get_multi_date_duration(df, dt):
    dd = df.reset_index()
    dd = dd[dd.date >= dt]
    # dd['couts'] = dd.groupby(['code'])['code'].transform('count')
    dd = dd.set_index(['code', 'date'])
    return dd


def get_multi_code_count(df, col='code'):
    dd = df.reset_index()
    dd['couts'] = dd.groupby([col])[col].transform('count')
    dd = dd.sort_values(by='couts', ascending=0)
    print('count dd.couts')
    dd = dd.set_index(['code', 'date'])
    return dd


def multindex_iloc(df, index):
    label = df.index.levels[0][index]
    return df.iloc[df.index.get_loc(label)]


def get_groupby_mean_median_close(dfs):

    groupd = dfs.groupby(level=[0])

    df = groupd['close'].agg(['median', 'mean'])
    df['close'] = groupd.tail(1).reset_index().set_index(['code'])['close']
    # dfs['mean'] = groupd['close'].agg('mean')
    # dfs['median'] = groupd['close'].agg('median')

    # dfs = dfs.fillna(0)
    # idx = pd.IndexSlice
    # mask = ( (dfs['mean'] > dfs['median'])
    #         & (dfs['close'] > dfs['mean'])
    #         )
    # df=dfs.loc[idx[mask, :]]

    df = df[(df['mean'] > df['median']) & (df['close'] > df['mean'])]

    # dt_low = None
    # if dl == 1:
    #     dfs = groupd.tail(1)
    #     print("dfs tail1")
    # else:
    #     dl = 30
    #     dindex = tdd.get_tdx_Exp_day_to_df(
    #         '999999', dl=dl).sort_index(ascending=False)
    #     dt = tdd.get_duration_price_date('999999', df=dindex)
    #     dt = dindex[dindex.index >= dt].index.values
    #     dt_low = dt[-1]
    #     dtlen = len(dt) if len(dt) >0 else 1
    #     dfs = groupd.tail(dtlen)
    #     print("dfs tail:%s dt:%s"%(dtlen,dt))
    #     dfs = get_multi_date_duration(dfs,dt[-1])
    return df

In [4]:
def get_roll_mean_all(
        single=True, tdx=False, app=True, duration=100, ma_250_l=1.02,
        ma_250_h=1.11, resample='d'):
    time_s = time.time()
    if not app and cct.get_file_size(block_path) > 100 and cct.creation_date_duration(block_path) == 0:
        print("It's Today Update")
        return True
    code_list = sina_data.Sina().market('all').index.tolist()
    code_list.extend(['999999', '399001', '399006'])
    print("all code:", len(code_list))
    if duration < 300:
        h5_fname = 'tdx_all_df' + '_' + str(300)
        h5_table = 'all' + '_' + str(300)
    else:
        h5_fname = 'tdx_all_df' + '_' + str(900)
        h5_table = 'all' + '_' + str(900)
    # df = tdd.search_Tdx_multi_data_duration('tdx_all_df_300', 'all_300', df=None,code_l=code_list, start='20150501', end=None, freq=None, col=None, index='date')
    df = tdd.search_Tdx_multi_data_duration(
        h5_fname, h5_table, df=None, code_l=code_list, start=None, end=None,
        freq=None, col=None, index='date')
    # df = tdd.search_Tdx_multi_data_duration(h5_fname, h5_table, df=None,code_l=code_list, start=None, end=None, freq=None, col=None, index='date',tail=1)

    code_uniquelist = df.index.get_level_values('code').unique()

    code_select = code_uniquelist[random.randint(0, len(code_uniquelist) - 1)]
    print(round(time.time() - time_s, 2), df.index.get_level_values(
        'code').unique().shape, code_select, df.loc[code_select].shape)
    # df.groupby(level=[0]),df.index.get_level_values(0)
    # len(df.index.get_level_values('code').unique())
    # df = df[~df.index.duplicated(keep='first')]

    # multiIndex_func = {'close': 'mean', 'low': 'min', 'high': 'max', 'volume': 'sum', 'open': 'first'}
    # cct.using_Grouper(df, freq='W', col={'close': 'last'})
    # *** TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'
    # roll_dl = duration

    if resample != 'd':
        print("resample:%s" % (resample.upper()))
        # ohlc_dict = {'volume': 'sum','high': 'max','low': 'min','close': 'last'}
        # {'low': 'min', 'close':'mean'}
        # df = df.groupby(level=0).resample(resample, level=1).agg({'close':'last'})
        # df = df.loc[:, ['open','close', 'low', 'high']].groupby(level=0).resample(resample, level=1).agg({'close':'last'})
        df = df.loc[:, ['close']].groupby(level=0).resample(
            resample, level=1).agg({'close': 'last'}).dropna()

        df = df.dropna()
        # df = df.loc[:, ['open','close', 'low', 'high']].groupby(level=0).resample(resample, level=1, how={'low': 'min', 'close':'mean'})
        # df = df.loc[:, ['open','close', 'low', 'high']].groupby(pd.Grouper(freq=resample)).transform(np.cumsum).resample(resample, how='ohlc')

        # df = df.loc[:, ['open','close', 'low', 'high']].groupby(pd.TimeGrouper(resample)).transform(np.cumsum).resample(resample, how='ohlc')
        # r1 = len(df.loc['000001'])
        # r2 = len(df.loc['999999'])
        # roll_dl = r1 if r1 < r2 else r2

        # df['upper'] = map(lambda x: round((1 + 11.0 / 100) * x, 1), df.ma10d)
        # df['lower'] = map(lambda x: round((1 - 9.0 / 100) * x, 1), df.ma10d)
        # df['ene'] = map(lambda x, y: round((x + y) / 2, 1), df.upper, df.lower)
        # dfs['amount%d'%cumdays] = groupd['amount'].apply(pd.rolling_mean, cumdays)
    # df.ix[df.index.levels[0]]
    # df.ix[df.index[len(df.index)-1][0]] #last row
    # dfs = tdd.search_Tdx_multi_data_duration(df=dfs,code_l=code_list, start='20170918', end='20170918', freq=None, col=None, index='date')

    # print dfs[:1],len(dfs)
    # groupd.agg({'low': 'min'})
    # '''idx mask filter'''
    # '''
    # h5a.write_hdf_db('all300', df, table='roll200', index=False,baseCount=500, append=False, MultiIndex=MultiIndex)
    print("use time:%s" % (round(time.time() - time_s, 2)))
    return df

In [5]:
resample = 'd'

df = get_roll_mean_all(
    single=False, tdx=True, app=True, duration=250, ma_250_l=1.02,
    ma_250_h=1.2, resample=resample)

all code: 4956
3.76 (4940,) 688670 (300, 6)
use time:3.86


# check HDF LastDate

In [26]:
df_tail1 = df.groupby(level=[0]).tail(1)
# df_tail1.index
df_tail1[df_tail1.index.get_level_values('date') == '2023-02-06'].shape
df_tail1[df_tail1.index.get_level_values('date') != '2023-02-06'].shape

(4882, 6)

(58, 6)

In [23]:
df.info()


df.loc['000001'][-1:]

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1427778 entries, ('002361', Timestamp('2021-11-10 00:00:00')) to ('600668', Timestamp('2023-02-06 00:00:00'))
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    1427778 non-null  float64
 1   high    1427778 non-null  float64
 2   low     1427778 non-null  float64
 3   close   1427778 non-null  float64
 4   vol     1427778 non-null  float64
 5   amount  1427778 non-null  float64
dtypes: float64(6)
memory usage: 71.1+ MB


,,open,high,low,close,vol,amount
code,date,,,,,,
600555,2022-02-07,1.19,1.19,1.18,1.19,17071997.0,20314982.0
600069,2020-05-12,1.02,1.02,1.02,1.02,3173600.0,3237072.0
600896,2022-02-07,4.42,4.53,4.31,4.31,8206900.0,35712268.0
000587,2023-01-19,1.00,1.01,0.93,0.93,74935400.0,72687338.0
600145,2022-01-28,1.00,1.04,1.00,1.00,81590508.0,82025531.0
600891,2021-03-19,1.07,1.07,1.07,1.07,7200.0,7704.0
603157,2022-02-07,1.33,1.35,1.33,1.33,3290800.0,4377789.0
600086,2020-11-12,1.00,1.00,0.96,0.98,10348363.0,10144548.0
600485,2020-04-20,1.35,1.39,1.30,1.39,55265436.0,76232728.0


,open,high,low,close,vol,amount
date,,,,,,
2023-02-06,14.1,14.13,13.86,14.0,131761010.0,1.838429e+09


In [7]:
app = True
tdx = True
duration = 300
ma_250_l = 1.02
ma_250_h = 1.2


time_s = time.time()
dfs = df.copy()
groupd = dfs.groupby(level=[0])
# groupd.['close']
# rollma = ['5','10','60','100','200']
# rollma = ['5','10','250']
# df.index.get_level_values('code')[0]
if resample.upper() == 'D' or resample.lower() == 'd':
    if duration < 300:
        rollma = ['10', '5', '26', '200']
    else:
        rollma = ['10', '5', '26', '250']

elif resample.upper() == 'W' or resample.lower() == 'w':
    rollma = ['5', '10']
else:
    rollma = ['5', '10']

# rollma.extend([str(duration)])

# import ipdb;ipdb.set_trace()
# df.loc['300130'][:2]

# dfs['mean'] = groupd['close'].agg('mean')
# dfs['median'] = groupd['close'].agg('median')

for da in rollma:
    cumdays = int(da)
    dfs['ma%d' % cumdays] = round(dfs['close'].rolling(cumdays).mean(), 2)
    if cumdays == 10:
        dfs['upper'] = dfs['ma%d' % cumdays].apply(
            lambda x: round((1 + 11.0 / 100) * x, 1))
        dfs['lower'] = dfs['ma%d' % cumdays].apply(
            lambda x: round((1 - 9.0 / 100) * x, 1))
        dfs['ene'] = list(map(lambda x, y: round(
            (x + y) / 2, 1), dfs['upper'], dfs['lower']))

In [8]:
# dfs.loc['301066']
dfs['ma51d'] = round(dfs['close'].rolling(5).mean(), 2)
print(dfs.loc['002987'])

             open   high    low  close         vol       amount   ma10  upper  \
date                                                                            
2021-11-10  20.71  21.03  20.19  20.39   4263023.0  122970656.0  64.08   71.1   
2021-11-11  20.51  20.51  20.11  20.42   2892321.0   82648080.0  59.31   65.8   
2021-11-12  20.30  20.62  20.18  20.44   2868141.0   82215088.0  54.48   60.5   
2021-11-15  20.45  20.99  20.45  20.91   4247483.0  123987232.0  49.72   55.2   
2021-11-16  20.78  20.84  20.26  20.34   3314718.0   95798112.0  44.76   49.7   
...           ...    ...    ...    ...         ...          ...    ...    ...   
2023-01-31  29.31  29.48  28.72  28.87   6420609.0  186121312.0  28.25   31.4   
2023-02-01  28.88  30.16  28.87  30.04   8889888.0  263173696.0  28.51   31.6   
2023-02-02  29.98  30.06  29.47  29.70   7633513.0  226921024.0  28.68   31.8   
2023-02-03  29.76  30.99  29.61  30.85  14687873.0  447803520.0  29.01   32.2   
2023-02-06  31.08  33.80  30

In [9]:
dfs = dfs.dropna()
single = False
dt_low = None
df_idx = None
if single:
    dfs_dtlen = groupd.tail(1)
    print("dfs tail1")
else:
    single = True
    dl = 30
    dindex = tdd.get_tdx_Exp_day_to_df(
        '999999', dl=dl, resample=resample).sort_index(ascending=False)
    dt = tdd.get_duration_price_date('999999', df=dindex)

    dt = dindex[dindex.index >= dt].index.values
    dt_low = dt[-1]
    print('dt_low:%s'%(dt_low))
    dtlen = len(dt) if len(dt) > 0 else 1
    dfs_dtlen = groupd.tail(dtlen)
    # import ipdb;ipdb.set_trace()
    df_idx = get_groupby_mean_median_close(dfs_dtlen)

    print(("dfs tail:%s dt:%s" % (dtlen, dt)))
    dfs_dtlen = get_multi_date_duration(dfs_dtlen, dt[-1])

# groupd2 = dfs.groupby(level=[0])
# dfs['ma%d'%cumdays] = groupd['close'].apply(pd.rolling_mean, cumdays)

# dfs.reset_index().groupby(['code'])['date'].transform('count')
# single = True

print("Low:%s"%(dt[-1]))
dfs_dtlen[:5]
dfs_dtlen[('ma%s') % (rollma[-1])][:2]
print(df_idx)

dt_low:2022-12-23
dfs tail:26 dt:['2023-02-06' '2023-02-03' '2023-02-02' '2023-02-01' '2023-01-31'
 '2023-01-30' '2023-01-20' '2023-01-19' '2023-01-18' '2023-01-17'
 '2023-01-16' '2023-01-13' '2023-01-12' '2023-01-11' '2023-01-10'
 '2023-01-09' '2023-01-06' '2023-01-05' '2023-01-04' '2023-01-03'
 '2022-12-30' '2022-12-29' '2022-12-28' '2022-12-27' '2022-12-26'
 '2022-12-23']
Low:2022-12-23


open     high      low    close          vol  \
code   date                                                          
999999 2022-12-23  3038.84  3061.87  3031.54  3045.87  192020305.0   
       2022-12-26  3048.20  3071.84  3047.35  3065.56  206503893.0   
       2022-12-27  3077.75  3098.08  3074.31  3095.57  222218322.0   
       2022-12-28  3088.62  3098.65  3079.43  3087.40  224554151.0   
       2022-12-29  3076.73  3086.00  3064.46  3073.70  215570676.0   

                         amount     ma10   upper   lower     ene      ma5  \
code   date                                                                 
999999 2022-12-23  2.333352e+11  3121.80  3465.2  2840.8  3153.0  3069.92   
       2022-12-26  2.554703e+11  3110.45  3452.6  2830.5  3141.6  3061.61   
       2022-12-27  2.679468e+11  3102.38  3443.6  2823.2  3133.4  3065.97   
       2022-12-28  2.605085e+11  3093.46  3433.7  2815.0  3124.3  3069.77   
       2022-12-29  2.539170e+11  3083.97  3423.2  2806.4  3114.8  3073.62   

                      ma26    ma250    ma51d  
code   date                                   
999999 2022-12-23  3134.87  3249.33  3069.92  
       2022-12-26  3133.65  3247.00  3061.61  
       2022-12-27  3134.06  3244.69  3065.97  
       2022-12-28  3134.00  3242.51  3069.77  
       2022-12-29  3133.11  3240.43  3073.62

code    date      
999999  2022-12-23    3249.33
        2022-12-26    3247.00
Name: ma250, dtype: float64

          median         mean    close
code                                  
000004     9.695     9.786538    10.06
000008     2.380     2.383846     2.45
000009    12.205    12.218077    12.27
000012     6.865     6.915000     7.08
000016     4.630     4.668846     4.74
...          ...          ...      ...
688789   169.390   175.131538   186.50
688798   104.420   108.481154   117.71
688799    33.360    33.527692    34.90
688981    41.770    41.878846    43.00
999999  3172.795  3179.056154  3238.70

[2930 rows x 3 columns]


# 基础处理

In [10]:
time_s = time.time()

def get_mask_df(dfs):
    dfs = dfs.fillna(0)
    idx = pd.IndexSlice
    # mask = (dfs[('ma%s')%(rollma[0])] > dfs[('ma%s')%(rollma[1])]) & (dfs[('ma%s')%(rollma[-1])] > 0) & (dfs[('close')] > dfs[('ma%s')%(rollma[0])])  & (dfs[('close')] > dfs[('ma%s')%(rollma[-1])])
    # mask = (dfs[('ma%s')%(rollma[0])] > dfs[('ma%s')%(rollma[1])]) & (dfs[('ma%s')%(rollma[-1])] > 0) & (dfs[('close')] > dfs[('ma%s')%(rollma[1])])  & (dfs[('close')] > dfs[('ma%s')%(rollma[-1])])
    # mask = (dfs[('ma%s')%(rollma[0])] > dfs[('ma%s')%(rollma[1])]) & (dfs[('ma%s')%(rollma[-1])] > 0) &  (dfs[('close')] > dfs[('ma%s')%(rollma[-1])])

    # mask = ( (dfs[('ma%s')%(rollma[0])] > 0) & (dfs[('ma%s')%(rollma[-1])] > 0) & (dfs[('close')] > dfs[('ma%s')%(rollma[-1])]) & (dfs[('close')] > dfs[('ma%s')%(rollma[0])]))
    # mask = ( (dfs[('ma%s')%(rollma[0])] > 0) & (dfs[('ma%s')%(rollma[-1])] > 0)
    #         & (dfs[('close')] > dfs[('ma%s')%(rollma[-1])]*ma_250_l)
    #         & (dfs[('close')] < dfs[('ma%s')%(rollma[-1])]*ma_250_h)
    #         & (dfs[('close')] > dfs[('ma%s')%(rollma[0])]))

    # & (dfs['mean'] > dfs['median'])
    # & (dfs['close'] > dfs['mean'])


    if len(rollma) > 1:

        mask = ((dfs[('ma%s') % (rollma[0])] > 0) & (dfs[('ma%s') % (rollma[-1])] > 0) &
                (dfs[('ma%s') % (rollma[0])] > dfs[('ma%s') % (rollma[-1])]) &
                (dfs[('close')] > dfs[('ma%s') % (rollma[0])]) &
                (dfs[('close')] > dfs[('ma%s') % (rollma[-1])]*ma_250_h) &
                ((dfs[('close')] > dfs['ene']) | (dfs[('close')] > dfs['upper']))
                )
    else:
        mask = ((dfs[('ma%s') % (rollma[0])] > 0) &
                (dfs[('close')] > dfs[('ma%s')%(rollma[0])]) &
                ((dfs[('close')] > dfs['ene']) | (dfs[('close')] > dfs['upper']) )
                )
    
    return dfs.loc[idx[mask, :]]

# mask = ((dfs[('close')] > dfs[('ma%s')%(rollma[-1])]))

df_mask = get_mask_df(dfs_dtlen)
print(df_mask.shape)
df_mask = get_multi_code_count(df_mask)
print((df_mask.couts[:5], df_mask.shape))

# import ipdb;ipdb.set_trace()
# df.sort_values(by='couts',ascending=0)
# groupd.first()[:2],groupd.last()[:2]
# groupd = df250.groupby(level=[0])
# '''
# groupd.transform(lambda x: x.iloc[-1])
# groupd.last()
# groupd.apply(lambda x: x.close > x.ma250)
# df.shape,df.sort_index(ascending=False)[:5]
# ?groupd.agg
# groupd = df.groupby(level=[0])
# groupd['close'].apply(pd.rolling_mean, 250, min_periods=1)
# ex:# Group df by df.platoon, then apply a rolling mean lambda function to df.casualties
# df.groupby('Platoon')['Casualties'].apply(lambda x:x.rolling(center=False,window=2).mean())

code_uniquelist = df_mask.index.get_level_values('code').unique()
code_select = code_uniquelist[random.randint(0, len(code_uniquelist) - 1)]

# if app:
#     print(round(time.time() - time_s, 2), 's', df.index.get_level_values(
#         'code').unique().shape, code_select, df.loc[code_select][-1:])

if single:
    # groupd = df.groupby(level=[0])
    if tdx:
        # block_path = tdd.get_tdx_dir_blocknew() + '060.blk'
        # if cct.get_work_time():
        #     codew = df[df.date == cct.get_today()].index.tolist()

        if dt_low is not None:

            groupd2 = df_mask.groupby(level=[0])
            df_mask = groupd2.tail(1)
            df_mask = df_mask.reset_index().set_index('code')
            # import ipdb;ipdb.set_trace()

            # df = df[(df.date >= dt_low) & (df.date <= cct.get_today())]
            dd = df_mask[(df_mask.date == dt_low)]
            # df = df[(df_mask.date >= cct.last_tddate(1))]
            df_mask = df_mask[(df_mask.date >= df_mask.date.max())]
            # import ipdb;ipdb.set_trace()
            print(("df:%s %s df_idx:%s" %
                  (len(df_mask), df_mask.index[:5], len(df_idx))))

            if df_idx is not None and len(df_mask) > 0 and len(df_idx) > 0:
                idx_set_ = [x for x in df_idx.index if x in df_mask.index]
                df_mask = df_mask.loc[idx_set_, :].dropna()
            print(("Main Down dd :%s MainUP df:%s couts std:%0.1f " %
                  (len(dd), len(df_mask), df_mask.couts.std())))
            # print df.date.mode()[0]
            df_mask = df_mask.sort_values(by='couts', ascending=1)
            df_mask = df_mask[df_mask.couts > df_mask.couts.std()]
            # df = df[(df.date >= df.date.mode()[0]) & (df.date <= cct.get_today())]
            codew = df_mask.index.tolist()

            if app:
                print(
                    "time: %s %s %s" %
                    (round(time.time() - time_s, 2),
                     'groupd2', len(df_mask)))

        else:
            df_mask = df_mask.reset_index().set_index('code')
            df_mask = df_mask[(df_mask.date >= cct.last_tddate(days=10)) &
                              (df_mask.date <= cct.get_today())]
            codew = df_mask.index.tolist()

        top_temp = tdd.get_sina_datadf_cnamedf(codew, df_mask)
        top_temp = top_temp[(~top_temp.index.str.contains(
            '688')) & (~top_temp.name.str.contains('ST'))]
        top_temp = top_temp.sort_values(by=['couts'], ascending=[0])
        codew = top_temp.index.tolist()

        # clean st and 688

    # df['date'] = df['date'].apply(lambda x:(x.replace('-','')))
    # df['date'] = df['date'].astype(int)
    # print df_mask.loc[code_select].T,df_mask.shape
#     MultiIndex = False
# else:
#     MultiIndex = True

(9850, 14)
count dd.couts
(code    date      
301239  2023-01-18    25
        2023-01-11    25
        2023-02-02    25
        2023-02-01    25
        2023-01-31    25
Name: couts, dtype: int64, (9850, 15))
df:195 Index(['688302', '301312', '002121', '001223', '600583'], dtype='object', name='code') df_idx:2930
Main Down dd :23 MainUP df:134 couts std:5.1 
time: 0.03 groupd2 62


In [11]:
dfs_dtlen.loc['002987'][:1]

,open,high,low,close,vol,amount,ma10,upper,lower,ene,ma5,ma26,ma250,ma51d
date,,,,,,,,,,,,,,
2022-12-23,25.12,25.92,25.07,25.6,4349418.0,111461984.0,26.5,29.4,24.1,26.8,25.83,28.62,24.24,25.83


# 最近30天筛选

In [28]:
# 截取最后5天数据
dfs_tail = dfs.groupby(level=[0]).tail(30)
# 还原多索引
# dfs_tail = dfs_tail.reset_index().set_index(['code','date'])
dfs_tail.loc['002987'][:1]

,open,high,low,close,vol,amount,ma10,upper,lower,ene,ma5,ma26,ma250,ma51d
date,,,,,,,,,,,,,,
2022-12-16,27.05,27.29,26.34,26.4,7254824.0,193081568.0,28.59,31.7,26.0,28.9,27.16,29.79,24.15,27.16


In [29]:
mask250 = ((dfs_tail[('ma%s') % (rollma[0])] > 0) & (dfs_tail[('ma%s') % (rollma[-1])] > 0) &
           (dfs_tail[('close')] > dfs_tail[('ma%s') % (rollma[0])]) &
            (dfs_tail[('close')] > dfs_tail[('ma%s') % (rollma[-1])]*1.03) &
              (dfs_tail[('close')] < dfs_tail[('ma%s') % (rollma[-1])]*1.1) &
                (dfs_tail[('close')] > dfs_tail[('ma26')]) &
           (dfs_tail[('close')] < dfs_tail[('ma26' )]*1.1)
           )
# idx = pd.IndexSlice
df_mask250 = dfs_tail.loc[idx[mask250, :]]
print(df_mask250.shape)
df_mask250 = get_multi_code_count(df_mask250)
print((df_mask250.couts[:1], df_mask250.shape))

groupd250 = df_mask250.groupby(level=[0])
df_mask250 = groupd250.tail(1)
print((df_mask250.couts[:5], df_mask250.shape))
print(df_mask.loc['002987'])

NameError: name 'idx' is not defined

# 默认年线筛选

In [41]:
top_temp.shape
top_temp.index

(47, 17)

Index(['002368', '002868', '600761', '301233', '002137', '001228', '002262',
       '301289', '002446', '002847', '301296', '003042', '301101', '301265',
       '603163', '002843', '301300', '000863', '300779', '002348', '600962',
       '600570', '601699', '301398', '002862', '301234', '300101', '002808',
       '300634', '603810', '000595', '301160', '600661', '002917', '603721',
       '603757', '002181', '301367', '002380', '002979', '600095', '000026',
       '300227', '603499', '001336', '000831', '002654'],
      dtype='object', name='code')

# 写入060

In [86]:
if app:
    print("Write blk:%s" % (block_path))
    hdf5_wri = cct.cct_raw_input("rewrite code [Y] or append [N]:")
    if len(hdf5_wri) == 0:
        pass
    elif hdf5_wri == 'y' or hdf5_wri == 'Y':
        append_status = False
    else:
        append_status = True
else:
    append_status = False

if hdf5_wri == 'y' and len(codew) > 10:
    cct.write_to_blocknew(
        block_path, codew, append_status, doubleFile=False, keep_last=0)
    print("write:%s block_path:%s" % (len(codew), block_path))
else:
    print("write error:%s block_path:%s" % (len(codew), block_path))

Write blk:D:\MacTools\WinTools\new_tdx\T0002\blocknew\060.blk


rewrite code [Y] or append [N]: n


write error:494 block_path:D:\MacTools\WinTools\new_tdx\T0002\blocknew\060.blk
